# Intro to Graph Convolutional Network

In [1]:
import numpy as np


In [2]:
adjacency_matrix = np.matrix('0,1,1,1;0,0,1,1;1,0,0,0;0,0,1,0',dtype=np.float32)
adjacency_matrix

matrix([[0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.]], dtype=float32)

In [3]:
feature_matrix = np.matrix('0,1,-1,0;5,3,2,1;1,0,0,0;1,1,0,0',dtype = np.float32)
feature_matrix

matrix([[ 0.,  1., -1.,  0.],
        [ 5.,  3.,  2.,  1.],
        [ 1.,  0.,  0.,  0.],
        [ 1.,  1.,  0.,  0.]], dtype=float32)

## Propagation


In [4]:
layer_0  = np.dot(adjacency_matrix,feature_matrix)
layer_0


matrix([[ 7.,  4.,  2.,  1.],
        [ 2.,  1.,  0.,  0.],
        [ 0.,  1., -1.,  0.],
        [ 1.,  0.,  0.,  0.]], dtype=float32)

## Including loops


In [5]:
identity = np.identity(adjacency_matrix.shape[0])
adjacency_matrix += identity
adjacency_matrix

matrix([[1., 1., 1., 1.],
        [0., 1., 1., 1.],
        [1., 0., 1., 0.],
        [0., 0., 1., 1.]], dtype=float32)

## Normalization


In [6]:
d = np.array(np.sum(adjacency_matrix,axis = 1)).T[0]
d

array([4., 3., 2., 2.], dtype=float32)

In [7]:
diagonal_mat = np.diag(d)
diagonal_mat

array([[4., 0., 0., 0.],
       [0., 3., 0., 0.],
       [0., 0., 2., 0.],
       [0., 0., 0., 2.]], dtype=float32)

In [8]:
inversed_diagonal_mat = np.linalg.inv(diagonal_mat)
inversed_diagonal_mat_half_symm = np.linalg.inv(diagonal_mat**0.5)
inversed_diagonal_mat_half_symm

array([[0.5       , 0.        , 0.        , 0.        ],
       [0.        , 0.57735026, 0.        , 0.        ],
       [0.        , 0.        , 0.70710677, 0.        ],
       [0.        , 0.        , 0.        , 0.70710677]], dtype=float32)

In [9]:
normalized_adj_mat = np.dot(inversed_diagonal_mat,adjacency_matrix)
normalized_adj_mat_half_symm = np.dot(inversed_diagonal_mat_half_symm,adjacency_matrix)
print("normalized_adj_mat_half_symm")
print(normalized_adj_mat_half_symm)
print("normalized_adj_mat_full_sym")
normalized_adj_mat_full_symm = inversed_diagonal_mat_half_symm*adjacency_matrix*inversed_diagonal_mat_half_symm
print(normalized_adj_mat_full_symm)

normalized_adj_mat_half_symm
[[0.5        0.5        0.5        0.5       ]
 [0.         0.57735026 0.57735026 0.57735026]
 [0.70710677 0.         0.70710677 0.        ]
 [0.         0.         0.70710677 0.70710677]]
normalized_adj_mat_full_sym
[[0.25       0.28867513 0.35355338 0.35355338]
 [0.         0.3333333  0.40824828 0.40824828]
 [0.35355338 0.         0.49999997 0.        ]
 [0.         0.         0.49999997 0.49999997]]


## Defining relu function

In [10]:
def relu(a):
    mask = a > 0
    return np.multiply(a,mask)

In [11]:
relu(inversed_diagonal_mat)

array([[0.25      , 0.        , 0.        , 0.        ],
       [0.        , 0.33333334, 0.        , 0.        ],
       [0.        , 0.        , 0.5       , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ]], dtype=float32)

In [12]:
relu(np.matrix('-1,4,2,-1;-22,3,-1,1'))

matrix([[0, 4, 2, 0],
        [0, 3, 0, 1]])

 # PyTorch implementation

In [13]:
from collections import namedtuple
from networkx import read_edgelist, set_node_attributes, to_numpy_matrix
from pandas import read_csv, Series
from numpy import array
import torch
import torch.nn as nn
import torch.optim as optim

# Data preprocessing 

In [14]:
DataSet = namedtuple(
    'DataSet',
    field_names=['X_train', 'y_train', 'X_test', 'y_test', 'network']
)

def load_karate_club():
    network = read_edgelist(
        'karate.edgelist',
        nodetype=int)

    attributes = read_csv(
        'karate.attributes.csv',
        index_col=['node'])

    for attribute in attributes.columns.values:
        set_node_attributes(
            network,
            values=Series(
                attributes[attribute],
                index=attributes.index).to_dict(),
            name=attribute
        )
  
    X_train,y_train = map(array, zip(*[
        ([node], data['role'] == 'Administrator')
        for node, data in network.nodes(data=True)
        if data['role'] in {'Administrator', 'Instructor'}
    ]))
  
    X_test, y_test = map(array, zip(*[
        ([node], data['community'] == 'Administrator')
        for node, data in network.nodes(data=True)
        if data['role'] == 'Member'
    ]))
 
    return DataSet(
        X_train, y_train,
        X_test, y_test,
        network)

In [35]:
zkc = load_karate_club()
X_train_flattened = torch.flatten(torch.from_numpy(zkc.X_train))
X_test_flattened = torch.flatten(torch.from_numpy(zkc.X_test))
y_train = torch.from_numpy(zkc.y_train).to(torch.float)
print(y_train)
A = to_numpy_matrix(zkc.network)
A = torch.from_numpy(np.array(A))
print(A)
print(X_train_flattened)

tensor([1., 0.])
tensor([[0., 1., 1.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.]], dtype=torch.float64)
tensor([ 0, 33], dtype=torch.int32)


# SpectralRule and LogisticRegressor Modules

In [16]:
class SpectralRule(nn.Module):
    
    def __init__(self,A,input_units,output_units,activation = 'relu'):
        
        super(SpectralRule,self).__init__()
        
        self.input_units = input_units
        self.output_units = output_units
        self.linear_layer = nn.Linear(self.input_units,self.output_units)
        
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        else:
            self.activation = nn.Identity()
        #Created Identity Matrix
        I = torch.eye(A.shape[1])
       
        #Adding self loops to the adjacency matrix
        A_hat = A + I
        A_hat = A_hat.to(torch.double)
       
        #Inverse degree Matrix
        D = torch.diag(torch.pow(torch.sum(A_hat,dim = 0),-0.5),0)
       
        #applying spectral rule
        self.A_hat = torch.matmul(torch.matmul(D,A_hat),D)
        self.A_hat.requires_grad = False
       
        
    def forward(self,X):
        
        #aggregation
        aggregation = torch.matmul(self.A_hat,X)
         
        #propagation
        linear_output = self.linear_layer(aggregation.to(torch.float))
      
        propagation = self.activation(linear_output)
          
        return propagation.to(torch.double)

class LogisticRegressor(nn.Module):
    
    def __init__(self,input_units,output_units=1):
        super(LogisticRegressor,self).__init__()
        
        self.Linear = nn.Linear(input_units,output_units,bias=True)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,X):
        linear_output = self.Linear(X.to(torch.float))
        return self.sigmoid(linear_output)
        
            
            
        
        
        
        

# Building the Base Model

In [17]:
hidden_layer_config = [(4,'tanh'),(2,'tanh')]

class Model(nn.Module):
    def __init__(self,A,hidden_layer_config,initial_input_size):
        super(Model,self).__init__()
        
        self.hidden_layer_config = hidden_layer_config
        
        self.moduleList = list()
        
        self.initial_input_size = initial_input_size
        
        for input_size,activation in hidden_layer_config:
            
            self.moduleList.append(SpectralRule(A,self.initial_input_size,input_size,activation))
            self.initial_input_size = input_size
        
        self.moduleList.append(LogisticRegressor(self.initial_input_size,1))
        
        self.sequentialModule = nn.Sequential(*self.moduleList)
        
    def forward(self,X):
        output = self.sequentialModule(X)
        return output
        

# Identity Matrix as feature

In [18]:
identity = torch.eye(A.shape[1])
identity = identity.to(torch.double)

In [19]:
model = Model(A,hidden_layer_config,identity.shape[1])

output = model(identity)

In [20]:
output

tensor([[0.3362],
        [0.3369],
        [0.3372],
        [0.3379],
        [0.3393],
        [0.3381],
        [0.3381],
        [0.3387],
        [0.3389],
        [0.3394],
        [0.3397],
        [0.3388],
        [0.3386],
        [0.3385],
        [0.3388],
        [0.3386],
        [0.3375],
        [0.3387],
        [0.3387],
        [0.3378],
        [0.3386],
        [0.3363],
        [0.3378],
        [0.3351],
        [0.3387],
        [0.3388],
        [0.3390],
        [0.3388],
        [0.3387],
        [0.3376],
        [0.3372],
        [0.3381],
        [0.3371],
        [0.3385]], grad_fn=<SigmoidBackward>)

In [21]:
output.shape

torch.Size([34, 1])

# Train Model

In [37]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(),lr = 0.1,momentum=0.9)

def train(model , epoch , criterion , optimizer , feature):
    for j in range(epoch):
        for i,node in enumerate(X_train_flattened):
            output = model(feature)[node]
            print("predicted "+str(i), output)
            ground_truth = torch.reshape(y_train[i],output.shape)
       
        
            optimizer.zero_grad() 
            
            loss = criterion(output,ground_truth)
            #\print("loss: ",loss.data)
            loss.backward()
            optimizer.step()
    
    torch.save(model.state_dict(),"./gcn.pth")
    

In [38]:
train(model,2000,criterion,optimizer,identity)

predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([8.0506e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([8.0501e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([8.0492e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([8.0480e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([8.0464e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([8.0447e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([8.0427e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([8.0405e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([8.0383e-05], 

predicted 1 tensor([7.8307e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.8281e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.8254e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.8228e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.8201e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.8175e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.8148e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.8122e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.8095e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], 

predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.6039e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.6014e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.5989e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.5964e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.5939e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.5914e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.5889e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.5864e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.5839e-05], 

predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.3709e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.3685e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.3662e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.3638e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.3615e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.3591e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.3568e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.3545e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.3521e-05], 

predicted 1 tensor([7.1429e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.1407e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.1384e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.1363e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.1340e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.1318e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.1296e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.1274e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([7.1252e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], 

...

predicted 1 tensor([5.1384e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.1372e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.1361e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.1349e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.1338e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.1326e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.1315e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.1304e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.1292e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], 

predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.0319e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.0308e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.0297e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.0286e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.0275e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.0264e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.0253e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.0242e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([5.0231e-05], 

predicted 1 tensor([4.9519e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.9508e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.9498e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.9487e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.9476e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.9466e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.9455e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.9445e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.9434e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], 

predicted 1 tensor([4.8755e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.8744e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.8734e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.8724e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.8713e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.8703e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.8693e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.8683e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.8672e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], 

predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.7914e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.7904e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.7894e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.7884e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.7874e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.7864e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.7854e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.7844e-05], grad_fn=<SelectBackward>)
predicted 0 tensor([0.9999], grad_fn=<SelectBackward>)
predicted 1 tensor([4.7834e-05], 

In [36]:
X_test_flattened

tensor([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 17, 19, 21, 31, 30,  9,
        27, 28, 32, 16, 14, 15, 18, 20, 22, 23, 25, 29, 24, 26],
       dtype=torch.int32)

In [45]:
def test(features):
    
    model = Model(A,hidden_layer_config,identity.shape[1])
    model.load_state_dict(torch.load("./gcn.pth"))
    model.eval()
    
    for i ,node in X_test_flattened:
        output = model(features)
        print(output)
    

In [46]:
test(identity)

tensor([[9.9995e-01],
        [9.9964e-01],
        [9.9694e-01],
        [9.7951e-01],
        [4.5573e-01],
        [9.7341e-01],
        [9.7357e-01],
        [8.5777e-02],
        [2.7491e-03],
        [4.6514e-01],
        [1.5498e-02],
        [3.5408e-02],
        [1.9951e-02],
        [1.4649e-02],
        [1.5898e-03],
        [1.7039e-02],
        [5.8058e-02],
        [1.5345e-03],
        [5.5873e-04],
        [1.7045e-03],
        [1.2674e-03],
        [9.6696e-01],
        [5.3141e-01],
        [9.9926e-01],
        [1.9633e-04],
        [2.1945e-04],
        [2.0759e-04],
        [2.1768e-04],
        [2.3519e-04],
        [3.5211e-04],
        [8.4390e-03],
        [6.6715e-05],
        [2.7550e-02],
        [4.7465e-05]], grad_fn=<SigmoidBackward>)
